In [1]:
from time import time
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('islcnnmodel.h5')


In [2]:

def prediction(pred):
    """
    Converts predicted class to corresponding character.
    """
    # Adjust ASCII mapping for predicted class
    if pred < 10:
        return chr(pred + 48)  # 0-9 for numbers
    else:
        return chr(pred + 65 - 10)  # A-Z for letters


In [3]:

def keras_predict(model, image):
    """
    Predict the class of the input image using the loaded model.
    """
    # Normalize image data and expand dimensions for batch
    pred_probab = model.predict(image)[0]
    pred_class = np.argmax(pred_probab)
    return max(pred_probab), pred_class



In [4]:

def preprocess_image(img):
    """
    Preprocess the image for the model.
    """
    # Resize to model input size (64x64)
    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_AREA)

    # Convert to RGB if needed
    if len(img.shape) == 2:  # Grayscale check
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Normalize and expand dimensions
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img



In [5]:

def crop_image(image, x, y, width, height):
    """
    Crop the region of interest (ROI) from the image.
    """
    return image[y:y + height, x:x + width]



In [6]:

def main():
    cam_capture = cv2.VideoCapture(0)  # Start video capture

    while True:
        ret, image_frame = cam_capture.read()
        if not ret:
            print("Failed to grab frame.")
            break
        
        # Crop the region of interest
        roi = crop_image(image_frame, 150, 150, 250, 250)

        # Preprocess for skin detection
        hsv_image = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        lower_boundary = np.array([0, 40, 30], dtype="uint8")
        upper_boundary = np.array([43, 255, 254], dtype="uint8")
        skin_mask = cv2.inRange(hsv_image, lower_boundary, upper_boundary)
        skin_mask = cv2.medianBlur(skin_mask, 5)
        skin = cv2.bitwise_and(roi, roi, mask=skin_mask)

        # Convert to grayscale and preprocess for model
        bw_image = cv2.cvtColor(skin, cv2.COLOR_BGR2GRAY)
        bw_image = cv2.GaussianBlur(bw_image, (5, 5), 0)

        # Prepare image for prediction
        processed_image = preprocess_image(bw_image)

        # Predict using the model
        pred_probab, pred_class = keras_predict(model, processed_image)
        curr_prediction = prediction(pred_class)

        # Display prediction on the video frame
        cv2.putText(
            image_frame, curr_prediction, (300, 150),
            cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA
        )

        # Draw the ROI rectangle
        cv2.rectangle(image_frame, (150, 150), (400, 400), (255, 255, 0), 3)

        # Show the video frames
        cv2.imshow("Frame", image_frame)
        cv2.imshow("Processed ROI", bw_image)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cam_capture.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 